# FeatherFace Baseline Training and Evaluation

This notebook reproduces the original FeatherFace training process following the author's instructions.

## Overview
- Model: FeatherFace with MobileNetV1 0.25x backbone
- Dataset: WIDERFace (auto-download)
- Expected Results: 0.49M parameters, 90.8% mAP
- Uses original training scripts for faithful reproduction

## 1. Installation and Environment Setup

In [1]:
# Setup paths - all paths are relative to the FeatherFace root directory
import os
import sys
from pathlib import Path

# Get the project root directory (parent of notebooks/)
PROJECT_ROOT = Path(os.path.abspath('..'))
print(f"Project root: {PROJECT_ROOT}")

# Change to project root for all operations
os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

Project root: /teamspace/studios/this_studio/FeatherFace
Working directory: /teamspace/studios/this_studio/FeatherFace


In [2]:
# Install project in editable mode
!pip install -e .

# Verify imports work
try:
    from models.retinaface import RetinaFace
    from data import cfg_mnet, WiderFaceDetection
    print("✓ Imports successful")
except ImportError as e:
    print(f"✗ Import error: {e}")

Obtaining file:///teamspace/studios/this_studio/FeatherFace
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for featherface (pyproject.toml) ... done
  Created wheel for featherface: filename=featherface-2.0.0-0.editable-py3-none-any.whl size=6766 sha256=203c33a378c6491af3cae85037ff68a841e5f60bfce5ece7fa981063ea2333fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-1pgsahhh/wheels/e5/25/0d/b1fa017cd463fed7d4ed29962d88edd331d2ec669cbd3734b5
Successfully built featherface
  Attempting uninstall: featherface
    Found existing installation: featherface 2.0.0
    Uninstalling featherface-2.0.0:
      Successfully uninstalled featherface-2.0.0
✓ Imports successful


In [3]:
# Verify imports and check GPU
import torch
import torchvision
import cv2
import numpy as np
import matplotlib.pyplot as plt
import gdown
import requests
import zipfile
import tarfile
import json
from datetime import datetime

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

Python version: 3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]
PyTorch version: 2.7.0+cu128
CUDA available: False


## 2. Dataset Download and Preparation

The dataset will be automatically downloaded when training starts. But we can prepare the directories.

In [4]:
# Check and create data directories
import os
from pathlib import Path

# Create necessary directories
data_dir = Path('data/widerface')
data_root=Path('data')
weights_dir = Path('weights')
results_dir = Path('results')


# WIDERFace download links
WIDERFACE_GDRIVE_ID = '11UGV3nbVv1x9IC--_tK3Uxf7hA6rlbsS'
WIDERFACE_URL = f'https://drive.google.com/uc?id={WIDERFACE_GDRIVE_ID}'

for dir_path in [data_dir, weights_dir, results_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"✓ Directory ready: {dir_path}")


def download_widerface():
    """Download WIDERFace dataset from Google Drive"""
    output_path = data_root/ 'widerface.zip'
    
    if not output_path.exists():
        print("Downloading WIDERFace dataset...")
        print("This may take several minutes depending on your connection.")
        
        try:
            gdown.download(WIDERFACE_URL, str(output_path), quiet=False)
            print(f"✓ Downloaded to {output_path}")
        except Exception as e:
            print(f"❌ Download failed: {e}")
            print("Please download manually from:")
            print(f"  {WIDERFACE_URL}")
            return False
    else:
        print(f"✓ Dataset already downloaded: {output_path}")
    
    return True

# Download dataset
if download_widerface():
    print("\n✅ Dataset download complete!")
else:
    print("\n❌ Please download the dataset manually.")

✓ Directory ready: data/widerface
✓ Directory ready: weights
✓ Directory ready: results
✓ Dataset already downloaded: data/widerface.zip

✅ Dataset download complete!


In [5]:
# Extract dataset
def extract_widerface():
    """Extract WIDERFace dataset"""
    zip_path = data_root / 'widerface.zip'
    
    if not zip_path.exists():
        print("❌ Dataset zip file not found. Please download first.")
        return False
    
    # Check if already extracted
    if (data_dir / 'train' / 'label.txt').absolute().exists() and \
       (data_dir / 'val' / 'wider_val.txt').absolute().exists():
        print("✓ Dataset already extracted")
        return True
    
    print("Extracting dataset...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_root)
        print("✓ Dataset extracted successfully")
        return True
    except Exception as e:
        print(f"❌ Extraction failed: {e}")
        return False

# Extract dataset
if extract_widerface():
    print("\n✅ Dataset ready for use!")
else:
    print("\n❌ Please extract the dataset manually.")

✓ Dataset already extracted

✅ Dataset ready for use!


In [6]:
# Verify dataset structure
def verify_dataset():
    """Verify WIDERFace dataset structure"""
    required_files = [
        data_dir / 'train' / 'label.txt',
        data_dir / 'val' / 'wider_val.txt'
    ]
    
    all_present = True
    for file_path in required_files:
        if file_path.absolute().exists():
            print(f"✓ Found: {file_path.absolute()}")
        else:
            print(f"✗ Missing: {file_path.absolute()}")
            all_present = False
    
    # Check for images
    for split in ['train', 'val']:
        img_dir = data_dir / split / 'images'
        if img_dir.exists():
            img_count = len(list(img_dir.glob('**/*.jpg')))
            print(f"✓ {split} images: {img_count} found")
        else:
            print(f"✗ {split} images directory not found")
            all_present = False
    
    return all_present

dataset_ready = verify_dataset()
print(f"\nDataset verification: {'PASSED ✅' if dataset_ready else 'FAILED ❌'}")

✓ Found: /teamspace/studios/this_studio/FeatherFace/data/widerface/train/label.txt
✓ Found: /teamspace/studios/this_studio/FeatherFace/data/widerface/val/wider_val.txt
✓ train images: 12880 found
✓ val images: 3226 found

Dataset verification: PASSED ✅


## 3. Download Pre-trained Weights

The model requires pre-trained MobileNetV1 0.25x weights.

In [7]:
# Pre-trained weights info
PRETRAIN_FILENAME = 'mobilenetV1X0.25_pretrain.tar'
pretrain_path = weights_dir / PRETRAIN_FILENAME

print("=== Pre-trained Weights Download Instructions ===")
print(f"\nWeights should be placed at: {pretrain_path.absolute()}")
print("\nDownload from:")
print("https://drive.google.com/open?id=1oZRSG0ZegbVkVwUd8wUIQx8W7yfZ_ki1")
print(f"\nSave as: {pretrain_path.relative_to('.')}")

if pretrain_path.exists():
    print(f"\n✓ Pre-trained weights found: {pretrain_path.relative_to('.')}")
else:
    print(f"\n✗ Pre-trained weights not found. Please download manually.")

=== Pre-trained Weights Download Instructions ===

Weights should be placed at: /teamspace/studios/this_studio/FeatherFace/weights/mobilenetV1X0.25_pretrain.tar

Download from:
https://drive.google.com/open?id=1oZRSG0ZegbVkVwUd8wUIQx8W7yfZ_ki1

Save as: weights/mobilenetV1X0.25_pretrain.tar

✓ Pre-trained weights found: weights/mobilenetV1X0.25_pretrain.tar


## 4. Model Configuration and Training Parameters

In [8]:
# Training parameters from original repository
TRAIN_CONFIG = {
    'network': 'mobile0.25',
    'num_workers': 16,  # Adjust based on your system
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'gamma': 0.1,
    'save_folder': 'weights/',
    'resume_net': './weights/mobilenet0.25_epoch_150.pth',  # Will be set to pretrained weights
    'resume_epoch': 151,
    'training_dataset': './data/widerface/train/label.txt'
}

# Additional config from cfg_mnet
from data import cfg_mnet

print("Training Configuration:")
for key, value in TRAIN_CONFIG.items():
    print(f"  {key}: {value}")
    
print("\nModel Configuration (cfg_mnet):")
print(f"  batch_size: {cfg_mnet['batch_size']}")
print(f"  epochs: {cfg_mnet['epoch']}")
print(f"  lr: {cfg_mnet['lr']}")
print(f"  gpu_train: {cfg_mnet['gpu_train']}")

Training Configuration:
  network: mobile0.25
  num_workers: 16
  momentum: 0.9
  weight_decay: 0.0005
  gamma: 0.1
  save_folder: weights/
  resume_net: ./weights/mobilenet0.25_epoch_150.pth
  resume_epoch: 151
  training_dataset: ./data/widerface/train/label.txt

Model Configuration (cfg_mnet):
  batch_size: 32
  epochs: 350
  lr: 0.001
  gpu_train: True


## 5. Training Process

We'll use the original train.py script with our configuration.

In [9]:
# Prepare training command
import subprocess
import sys

# Build command arguments based on train.py argparse
train_args = [
    sys.executable, 'train.py',
    '--training_dataset', TRAIN_CONFIG['training_dataset'],
    '--network', TRAIN_CONFIG['network'],
    '--num_workers', str(TRAIN_CONFIG['num_workers']),
    '--momentum', str(TRAIN_CONFIG['momentum']),
    '--weight_decay', str(TRAIN_CONFIG['weight_decay']),
    '--gamma', str(TRAIN_CONFIG['gamma']),
    '--save_folder', TRAIN_CONFIG['save_folder']
]

# Add resume options if specified
if TRAIN_CONFIG['resume_net']:
    train_args.extend(['--resume_net', TRAIN_CONFIG['resume_net']])
    train_args.extend(['--resume_epoch', str(TRAIN_CONFIG['resume_epoch'])])

print("Training command:")
print(' '.join(train_args))

print("\nNote: The training script uses configuration from data/config.py for:")
print("  - Learning rate (lr)")
print("  - Batch size")
print("  - Number of epochs")
print("  - GPU settings")
print("  - Pretrain flag")

Training command:
/home/zeus/miniconda3/envs/cloudspace/bin/python train.py --training_dataset ./data/widerface/train/label.txt --network mobile0.25 --num_workers 16 --momentum 0.9 --weight_decay 0.0005 --gamma 0.1 --save_folder weights/ --resume_net ./weights/mobilenet0.25_epoch_150.pth --resume_epoch 151

Note: The training script uses configuration from data/config.py for:
  - Learning rate (lr)
  - Batch size
  - Number of epochs
  - GPU settings
  - Pretrain flag


### Check Pre-trained Weights

The training script expects pre-trained MobileNetV1 weights if cfg_mnet['pretrain'] is True.

In [10]:
# Check for pretrained weights
pretrain_path = Path('weights/mobilenetV1X0.25_pretrain.tar')
if pretrain_path.exists():
    print(f"✓ Pre-trained weights found: {pretrain_path}")
else:
    print(f"✗ Pre-trained weights not found: {pretrain_path}")
    print("\nDownload from: https://drive.google.com/open?id=1oZRSG0ZegbVkVwUd8wUIQx8W7yfZ_ki1")
    print("Save to: weights/mobilenetV1X0.25_pretrain.tar")

✓ Pre-trained weights found: weights/mobilenetV1X0.25_pretrain.tar


In [11]:
# Option 1: Run training directly (recommended for full training)
# Uncomment to run:
#result = subprocess.run(train_args, capture_output=True, text=True)
#print(result.stdout)
#if result.stderr:
#    print("Errors:", result.stderr)

# Option 2: Show manual command for terminal execution
#print("\n=== To train manually in terminal ===")
#print("Navigate to project root and run:")
#print(' '.join(train_args).replace(sys.executable, 'python'))

## 6. Model Evaluation on WIDERFace

After training completes, we evaluate the model using test_widerface.py

In [12]:
# Check for trained model
import glob

# Find the latest checkpoint
checkpoints = sorted(glob.glob('weights/mobilenet0.25_*.pth'))
if checkpoints:
    latest_checkpoint = checkpoints[-1]
    print(f"Found checkpoint: {latest_checkpoint}")
else:
    print("No checkpoints found. Please train the model first.")
    latest_checkpoint = None

Found checkpoint: weights/mobilenet0.25_epoch_90.pth


In [13]:
# Evaluation parameters
EVAL_CONFIG = {
    'trained_model':  'weights/mobilenet0.25_Final.pth',
    'network': 'mobile0.25',
    'confidence_threshold': 0.02,
    'top_k': 5000,
    'nms_threshold': 0.4,
    'keep_top_k': 750,
    'save_folder': './widerface_evaluate/widerface_txt/',
    'dataset_folder': './data/widerface/val/images/',
    'origin_size': 'True',  # String value expected by argparse
    'save_image': True,
    'vis_thres': 0.5,
    'cpu': False  # Set to True if no GPU available
}

print("Evaluation Configuration:")
for key, value in EVAL_CONFIG.items():
    print(f"  {key}: {value}")

Evaluation Configuration:
  trained_model: weights/mobilenet0.25_Final.pth
  network: mobile0.25
  confidence_threshold: 0.02
  top_k: 5000
  nms_threshold: 0.4
  keep_top_k: 750
  save_folder: ./widerface_evaluate/widerface_txt/
  dataset_folder: ./data/widerface/val/images/
  origin_size: True
  save_image: True
  vis_thres: 0.5
  cpu: False


In [14]:
# Build evaluation command
eval_args = [
    sys.executable, 'test_widerface.py',
    '-m', EVAL_CONFIG['trained_model'],
    '--network', EVAL_CONFIG['network'],
    '--confidence_threshold', str(EVAL_CONFIG['confidence_threshold']),
    '--top_k', str(EVAL_CONFIG['top_k']),
    '--nms_threshold', str(EVAL_CONFIG['nms_threshold']),
    '--keep_top_k', str(EVAL_CONFIG['keep_top_k']),
    '--save_folder', EVAL_CONFIG['save_folder'],
    '--dataset_folder', EVAL_CONFIG['dataset_folder'],
    '--vis_thres', str(EVAL_CONFIG['vis_thres']),
    '--origin_size', EVAL_CONFIG['origin_size']  # Pass as string value
]

# Add optional flags
if EVAL_CONFIG['save_image']:
    eval_args.append('--save_image')
    
if EVAL_CONFIG['cpu']:
    eval_args.append('--cpu')

print("Evaluation command:")
print(' '.join(eval_args))

Evaluation command:
/home/zeus/miniconda3/envs/cloudspace/bin/python test_widerface.py -m weights/mobilenet0.25_Final.pth --network mobile0.25 --confidence_threshold 0.02 --top_k 5000 --nms_threshold 0.4 --keep_top_k 750 --save_folder ./widerface_evaluate/widerface_txt/ --dataset_folder ./data/widerface/val/images/ --vis_thres 0.5 --origin_size True --save_image


In [15]:
# Debug: Check evaluation arguments
print("=== Evaluation Arguments Debug ===")
for i, arg in enumerate(eval_args):
    print(f"{i}: '{arg}'")
    
print("\n=== Command as string ===")
print(' '.join(eval_args))

=== Evaluation Arguments Debug ===
0: '/home/zeus/miniconda3/envs/cloudspace/bin/python'
1: 'test_widerface.py'
2: '-m'
3: 'weights/mobilenet0.25_Final.pth'
4: '--network'
5: 'mobile0.25'
6: '--confidence_threshold'
7: '0.02'
8: '--top_k'
9: '5000'
10: '--nms_threshold'
11: '0.4'
12: '--keep_top_k'
13: '750'
14: '--save_folder'
15: './widerface_evaluate/widerface_txt/'
16: '--dataset_folder'
17: './data/widerface/val/images/'
18: '--vis_thres'
19: '0.5'
20: '--origin_size'
21: 'True'
22: '--save_image'

=== Command as string ===
/home/zeus/miniconda3/envs/cloudspace/bin/python test_widerface.py -m weights/mobilenet0.25_Final.pth --network mobile0.25 --confidence_threshold 0.02 --top_k 5000 --nms_threshold 0.4 --keep_top_k 750 --save_folder ./widerface_evaluate/widerface_txt/ --dataset_folder ./data/widerface/val/images/ --vis_thres 0.5 --origin_size True --save_image


### Alternative: Run without origin_size parameter

If you get an error with origin_size, try removing it from the command:

In [16]:
# Alternative evaluation args without origin_size
eval_args_no_origin = [
    sys.executable, 'test_widerface.py',
    '-m', EVAL_CONFIG['trained_model'],
    '--network', EVAL_CONFIG['network'],
    '--confidence_threshold', str(EVAL_CONFIG['confidence_threshold']),
    '--top_k', str(EVAL_CONFIG['top_k']),
    '--nms_threshold', str(EVAL_CONFIG['nms_threshold']),
    '--keep_top_k', str(EVAL_CONFIG['keep_top_k']),
    '--save_folder', EVAL_CONFIG['save_folder'],
    '--dataset_folder', EVAL_CONFIG['dataset_folder'],
    '--vis_thres', str(EVAL_CONFIG['vis_thres'])
]

if EVAL_CONFIG['save_image']:
    eval_args_no_origin.append('--save_image')
if EVAL_CONFIG['cpu']:
    eval_args_no_origin.append('--cpu')

print("Alternative command (no origin_size):")
print(' '.join(eval_args_no_origin).replace(sys.executable, 'python'))

Alternative command (no origin_size):
python test_widerface.py -m weights/mobilenet0.25_Final.pth --network mobile0.25 --confidence_threshold 0.02 --top_k 5000 --nms_threshold 0.4 --keep_top_k 750 --save_folder ./widerface_evaluate/widerface_txt/ --dataset_folder ./data/widerface/val/images/ --vis_thres 0.5 --save_image


In [17]:
# Option 1: Run evaluation directly (recommended)
# Uncomment to run:
#result = subprocess.run(eval_args_no_origin, capture_output=True, text=True)
#print(result.stdout)
#if result.stderr:
#    print("Errors:", result.stderr)

# Option 2: Test with origin_size (if the above doesn't work)
# result = subprocess.run(eval_args, capture_output=True, text=True)
# print(result.stdout)
# if result.stderr:
#     print("Errors:", result.stderr)

# Option 3: Show manual command for terminal execution
print("\n=== To evaluate manually in terminal ===")
print("Navigate to project root and run (recommended):")
print(' '.join(eval_args_no_origin).replace(sys.executable, 'python'))

# The evaluation will generate prediction files in widerface_evaluate/widerface_txt/


=== To evaluate manually in terminal ===
Navigate to project root and run (recommended):
python test_widerface.py -m weights/mobilenet0.25_Final.pth --network mobile0.25 --confidence_threshold 0.02 --top_k 5000 --nms_threshold 0.4 --keep_top_k 750 --save_folder ./widerface_evaluate/widerface_txt/ --dataset_folder ./data/widerface/val/images/ --vis_thres 0.5 --save_image


In [18]:
# Ready-to-use evaluation command for copy-paste
print("=== Copy-paste ready command ===")
cmd = ' '.join(eval_args_no_origin).replace(sys.executable, 'python')
print(cmd)

# Example expected output:
# python test_widerface.py -m weights/mobilenet0.25_Final.pth --network mobile0.25 ...

# To run evaluation with subprocess (uncomment):
# result = subprocess.run(eval_args_no_origin, capture_output=True, text=True)
# if result.returncode == 0:
#     print("Success!")
#     print(result.stdout)
# else:
#     print("Error occurred:")
#     print(result.stderr)

=== Copy-paste ready command ===
python test_widerface.py -m weights/mobilenet0.25_Final.pth --network mobile0.25 --confidence_threshold 0.02 --top_k 5000 --nms_threshold 0.4 --keep_top_k 750 --save_folder ./widerface_evaluate/widerface_txt/ --dataset_folder ./data/widerface/val/images/ --vis_thres 0.5 --save_image


### Computing mAP Scores

After running test_widerface.py, use the evaluation tools to compute mAP:

In [19]:
# After evaluation, compute mAP scores
print("=== Steps to compute mAP ===")
print("1. Run test_widerface.py (generates prediction txt files)")
print("2. Navigate to widerface_evaluate/")
print("3. Run evaluation script:")
print("   cd widerface_evaluate")
print("   python evaluation.py")
print("\nThis will output:")
print("- Easy Val AP: xx.x%")
print("- Medium Val AP: xx.x%")
print("- Hard Val AP: xx.x%")

=== Steps to compute mAP ===
1. Run test_widerface.py (generates prediction txt files)
2. Navigate to widerface_evaluate/
3. Run evaluation script:
   cd widerface_evaluate
   python evaluation.py

This will output:
- Easy Val AP: xx.x%
- Medium Val AP: xx.x%
- Hard Val AP: xx.x%


In [20]:
# Build evaluation command with proper arguments - FIXED VERSION
import os
from pathlib import Path

# Verify paths exist first
pred_dir = Path('./widerface_evaluate/widerface_txt')
gt_dir = Path('./widerface_evaluate/eval_tools/ground_truth')

print("=== Path Verification ===")
print(f"Predictions directory: {pred_dir.absolute()}")
print(f"Exists: {pred_dir.exists()}")
print(f"Ground truth directory: {gt_dir.absolute()}")
print(f"Exists: {gt_dir.exists()}")

# Create directories if they don't exist
pred_dir.mkdir(parents=True, exist_ok=True)
gt_dir.mkdir(parents=True, exist_ok=True)

# Build correct evaluation command with required arguments
eval_wider_args = [
    sys.executable, 'widerface_evaluate/evaluation.py',
    '-p', './widerface_evaluate/widerface_txt',    # Predictions path
    '-g', './widerface_evaluate/eval_tools/ground_truth'  # Ground truth path
]

print("\n=== Corrected Evaluation Command ===")
print("Evaluation command:")
print(' '.join(eval_wider_args))

# Check if prediction files exist before running evaluation
if pred_dir.exists() and any(pred_dir.rglob('*.txt')):
    print("\n✓ Prediction files found, ready to evaluate")
    print("Uncomment the lines below to run evaluation:")
    print("# result = subprocess.run(eval_wider_args, capture_output=True, text=True)")
    print("# print(result.stdout)")
    print("# if result.stderr:")
    print("#     print('Errors:', result.stderr)")
else:
    print("\n❌ No prediction files found!")
    print("Please run test_widerface.py first to generate predictions:")
    print("Example:")
    print("python test_widerface.py -m weights/mobilenet0.25_Final.pth --network mobile0.25 --save_folder ./widerface_evaluate/widerface_txt/")

=== Path Verification ===
Predictions directory: /teamspace/studios/this_studio/FeatherFace/widerface_evaluate/widerface_txt
Exists: True
Ground truth directory: /teamspace/studios/this_studio/FeatherFace/widerface_evaluate/eval_tools/ground_truth
Exists: True

=== Corrected Evaluation Command ===
Evaluation command:
/home/zeus/miniconda3/envs/cloudspace/bin/python widerface_evaluate/evaluation.py -p ./widerface_evaluate/widerface_txt -g ./widerface_evaluate/eval_tools/ground_truth

✓ Prediction files found, ready to evaluate
Uncomment the lines below to run evaluation:
# result = subprocess.run(eval_wider_args, capture_output=True, text=True)
# print(result.stdout)
# if result.stderr:
#     print('Errors:', result.stderr)


In [24]:
result = subprocess.run(eval_wider_args, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print('Errors:', result.stderr)

event_dir
./widerface_evaluate/widerface_txt/0--Parade
event_dir
./widerface_evaluate/widerface_txt/1--Handshaking
event_dir
./widerface_evaluate/widerface_txt/10--People_Marching
event_dir
./widerface_evaluate/widerface_txt/11--Meeting
event_dir
./widerface_evaluate/widerface_txt/12--Group
event_dir
./widerface_evaluate/widerface_txt/13--Interview
event_dir
./widerface_evaluate/widerface_txt/14--Traffic
event_dir
./widerface_evaluate/widerface_txt/15--Stock_Market
event_dir
./widerface_evaluate/widerface_txt/16--Award_Ceremony
event_dir
./widerface_evaluate/widerface_txt/17--Ceremony
event_dir
./widerface_evaluate/widerface_txt/18--Concerts
event_dir
./widerface_evaluate/widerface_txt/19--Couple
event_dir
./widerface_evaluate/widerface_txt/2--Demonstration
event_dir
./widerface_evaluate/widerface_txt/20--Family_Group
event_dir
./widerface_evaluate/widerface_txt/21--Festival
event_dir
./widerface_evaluate/widerface_txt/22--Picnic
event_dir
./widerface_evaluate/widerface_txt/23--Shopper

## 7. Model Analysis

Let's analyze the model architecture and count parameters.

In [21]:
# Load and analyze model
import torch
from models.retinaface import RetinaFace
from data import cfg_mnet

# Create model
net = RetinaFace(cfg=cfg_mnet, phase='test')

# Count parameters
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total, trainable = count_parameters(net)
print(f"Total parameters: {total:,} ({total/1e6:.2f}M)")
print(f"Trainable parameters: {trainable:,} ({trainable/1e6:.2f}M)")

# Expected: ~0.49M parameters

Total parameters: 592,371 (0.59M)
Trainable parameters: 592,371 (0.59M)


In [22]:
# Analyze model architecture by module
print("\n=== Model Architecture Analysis ===")
for name, module in net.named_children():
    params = sum(p.numel() for p in module.parameters())
    print(f"{name}: {params:,} parameters ({params/1e6:.3f}M)")


=== Model Architecture Analysis ===
body: 213,072 parameters (0.213M)
bacbkbone_0_cbam: 680 parameters (0.001M)
relu_0: 0 parameters (0.000M)
bacbkbone_1_cbam: 2,284 parameters (0.002M)
relu_1: 0 parameters (0.000M)
bacbkbone_2_cbam: 8,564 parameters (0.009M)
relu_2: 0 parameters (0.000M)
bifpn: 112,606 parameters (0.113M)
bif_cbam_0: 680 parameters (0.001M)
bif_relu_0: 0 parameters (0.000M)
bif_cbam_1: 680 parameters (0.001M)
bif_relu_1: 0 parameters (0.000M)
bif_cbam_2: 680 parameters (0.001M)
bif_relu_2: 0 parameters (0.000M)
ssh1: 77,655 parameters (0.078M)
ssh2: 77,655 parameters (0.078M)
ssh3: 77,655 parameters (0.078M)
ssh1_cs: 4,640 parameters (0.005M)
ssh2_cs: 4,640 parameters (0.005M)
ssh3_cs: 4,640 parameters (0.005M)
ClassHead: 780 parameters (0.001M)
BboxHead: 1,560 parameters (0.002M)
LandmarkHead: 3,900 parameters (0.004M)


## 8. Results Summary

After running the evaluation, compare with expected baseline results:

In [23]:
# Expected baseline results
baseline_results = {
    'Model': 'FeatherFace (MobileNetV1 0.25x)',
    'Parameters': '0.49M',
    'WIDERFace Easy': '90.8%',
    'WIDERFace Medium': '88.2%',
    'WIDERFace Hard': '77.2%',
    'Average mAP': '85.4%'
}

print("=== Expected Baseline Results ===")
for metric, value in baseline_results.items():
    print(f"{metric}: {value}")

print("\n=== Your Results ===")
print("Check results/ directory for evaluation outputs")
print("Use evaluation tools to compute mAP scores")

=== Expected Baseline Results ===
Model: FeatherFace (MobileNetV1 0.25x)
Parameters: 0.49M
WIDERFace Easy: 90.8%
WIDERFace Medium: 88.2%
WIDERFace Hard: 77.2%
Average mAP: 85.4%

=== Your Results ===
Check results/ directory for evaluation outputs
Use evaluation tools to compute mAP scores


## 9. Next Steps - FeatherFace V2

With baseline established, we can proceed to Phase 02 for FeatherFace V2 development:

1. **Architecture Optimizations**:
   - Replace standard convolutions with grouped/depthwise convolutions
   - Implement CBAM++ attention modules
   - Optimize FPN with lightweight operations

2. **Target Specifications**:
   - Parameters: 0.25M (50% reduction)
   - Performance: 92%+ mAP (1.2% improvement)
   - Maintain real-time inference speed

3. **Implementation Plan**:
   - Create new model variant in models/
   - Implement optimized modules in layers/
   - Train with enhanced augmentation
   - Fine-tune hyperparameters